In [5]:
import pandas as pd
seg_df=pd.read_csv('Seguro-Trimmed.csv')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
seg_df.replace(-1,np.NaN,inplace=True)

seg_df.isnull().sum()
seg_df.columns
seg_df.shape

y=seg_df['target']
seg_df.drop(columns='target',inplace=True)
y



0        0
1        0
2        0
3        0
4        0
        ..
54447    0
54448    0
54449    0
54450    0
54451    1
Name: target, Length: 54452, dtype: int64

In [6]:
miss_val_df= pd.DataFrame({'column_name':seg_df.columns,
                          'missing_value_count':seg_df.isnull().sum(),
                          'missing_percentage':seg_df.isnull().sum()/seg_df.shape[0]*100})
#miss_val_df

miss_val_df=miss_val_df.sort_values(by='missing_percentage',ascending=False)
#miss_val_df

drop_columns=list(miss_val_df[miss_val_df['missing_percentage']>50]['column_name'])

seg_df.drop(columns=drop_columns,inplace=True)

seg_df.shape

seg_int_float = [i for i in seg_df.columns if seg_df[i].dtype == 'int64' or seg_df[i].dtype == 'float64']
len(seg_int_float)
drop_columns

seg_object = [i for i in seg_df.columns if seg_df[i].dtype == object]
seg_object
len(seg_object)
seg_df.shape
length = len(seg_object) + len(seg_int_float)
length

for i in seg_int_float:
    seg_df[i].fillna(seg_df[i].mean(),inplace=True)
for i in seg_int_float:
    seg_df[i].fillna(seg_df[i].mode()[0],inplace=True)
    
len(seg_df.columns)


seg_df.isnull().sum()
seg_df.shape

(54452, 57)

In [7]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(seg_df,y,test_size=0.2,random_state=26)
for col in X_train.columns:
        if X_train[col].dtype==object:
            X_train[col]=X_train[col].fillna(X_train[col].mode()[0]).astype(object)
            X_test[col]=X_test[col].fillna(X_train[col].mode()[0]).astype(object)
        else:
            X_train[col]=X_train[col].fillna(X_train[col].mean())
            X_test[col]=X_test[col].fillna(X_train[col].mean())
            
X_train
Y_train

X_test
Y_test

X_train.shape

(43561, 57)

In [63]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train,Y_train)
y_pred = logreg.predict(X_test)


comp_df = pd.DataFrame({'Actual_values': Y_test,'Predicted':y_pred})
comp_df

y_pred_proba = logreg.predict_proba(X_test)[::,1]
y_pred_proba

from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,roc_auc_score,roc_curve
confusion_matrix(Y_test,y_pred)


array([[10501,     0],
       [  390,     0]], dtype=int64)

In [65]:
!pip install imblearn

  Using cached https://files.pythonhosted.org/packages/81/a7/4179e6ebfd654bd0eac0b9c06125b8b4c96a9d0a8ff9e9507eb2a26d2d7e/imblearn-0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/eb/aa/eba717a14df36f0b6f000ebfaf24c3189cd7987130f66cc3513efead8c2a/imbalanced_learn-0.6.1-py3-none-any.whl


In [8]:
import imblearn
from imblearn.over_sampling import SMOTE 

smote=SMOTE(sampling_strategy=0.40)
Xsmote,Ysmote=smote.fit_resample(X_train,Y_train)


In [10]:
X_train.shape, Xsmote.shape
Ysmote.value_counts(normalize=True)

0    0.714288
1    0.285712
Name: target, dtype: float64